In [1]:
import re
import os
import numpy as np
import glob
from tqdm import tqdm
import pandas as pd
from subprocess import Popen, PIPE, call, check_call
import datetime
import shutil
from datetime import datetime
import time

from help_functions import * #user def
from lut import tests
from itertools import compress


## General defines, paths etc

In [2]:
transcript_path = os.environ['NOELV']
grlib_path = os.environ['GRLIB']
riscv_dv = os.environ['r_dv']+"/"
now = datetime.now()
current_time = now.strftime("%Y-%m-%d")#_%H:%M:%S"
finish_after = int(1e5)
boot_loader_length = 41
instr_cnt = finish_after+boot_loader_length+2 #+41 to account for bootloader, +2 for counter trailing

In [51]:
#Each level correponds to one cell
settings = {
    "gen_test":{
        "errors":["You do not have a valid license to run Riviera-PRO","Error: Simulation initialization failed."],
        "paths":{
            "test":riscv_dv+"out/ovpsim_sim/*.log",
            "srec_test":riscv_dv+"/out/asm_test/*.srec",
        },
        "test_param" : "python3 run.py --custom_target target/rv64_noelv/ --iss ovpsim\
             --simulator riviera --isa rv64gc --mabi lp64 -o "+riscv_dv+'out',
        "process_param" : 'cd '+riscv_dv+' && source ~/.bashrc && module load aldec/riviera/2021.10 &&',
        "timeout" : False,
        "arg_timeout" : "-gdisas=1 -gfinish_after_en=1 -gfinish_after="+str(instr_cnt),
        "arg_no_timeout" : "-gdisas=1 -gfinish_after_en=1 -gfinish_after="+str(int(instr_cnt*1.1)),
        "iteration":-1, #will be set when run
    },
    "run_rtl":{
        "paths":{
            "cp_dest":transcript_path+'ram.srec',
            "rtl_log":riscv_dv+"out/rtl_log/",
            "srec":riscv_dv+"out/asm_test/",
            "transcript":transcript_path+"transcript",
            "Makefile":grlib_path+"/designs/noelv-generic/Makefile"
            
        },
        "process_param":'cd '+transcript_path+' && source ~/.bashrc && module add mentor/questasim/2021.3 && make sim-run',
        "test_sucess":["# ** Failure: *** IU in error mode, simulation halted ***","Test finished after","** Failure: Assertion violation."],
    },
    "parse_rtl":{
                "reg_ex":{
                    "lookup":re.compile("#\s*\d*\s*ns\s*:\sC\d I\d : \d*\s*\[\d\] @0x[0-9a-fA-F]{16} \(0x[0-9a-fA-F]{4,8}\)\s*.*"),
                    "pc":re.compile("(?<=@0x)[0-9a-fA-F]{16}"),
                    # "pc_csr_status_tval":re.compile("[0-9a-fA-F]{16}|X{16}"),
                    "binary" : re.compile("(?<=(\(0x))[a0-z9]{4,8}(?=\))"), #opcode
                    #"instruction" : re.compile("[a-z.]{2,9}(?=(\s\w{2,3},))|(\w*\.[a-z])|nop|mret|ecall|fence|unknown instruction|ebreak"),
                    #"instr_str" : re.compile("\w*\s\w*,\s\w*,\s-?\w*(?=\s*W)|\w*(?=\s*W)|\w*\s\w*,\s\w*(?=\s*W)"), #ugly
                    "instr_str" : re.compile("(?<!\])(?<=\)\s)[a-z0-9.,\- ()]*(?=\s*W)"),
                    # "gpr" : re.compile("(?<=([a-z]{2}\s))\w{2,3}(?=,)"),
                    "gpr" : re.compile("(?<=\[)[a-z0-9]*\s*=0x[0-9a-fA-FX]{16}(?=\]\[\d\]\sWF?)"),
                    #"op1" : re.compile("(?<=,\s)\w{1,8}((?=,)|(?=\s*W))|0\(\w{2,3}\)"), 
                    # "op2" : re.compile("(\w*,\s\w*,\s-?\w*)"), #use split to get op2, don't have a nicer regex atm
                    "isFloat":re.compile("^f(?!ence)"),
                    "isFloatReg":re.compile("^f[sta]\d{1,2}"),
                    "csr" : re.compile("(?<=\[)[a-z0-9]*\s*=0x[0-9a-fA-F]{16}(?=\]\[\d\] IPC)"),
                    "FPU_id":re.compile("(?<=# FPU 0x)[0-9a-fA-F]{2}"),
                    "FPU_val":re.compile("(?<=# FPU 0x[0-9a-fA-F]{2} ).*"),
                    "mode" : re.compile("(?<=PRV\[)\d*"),
                    "valid" : re.compile("(?<=\[)\d(?=\]\s@)"),
                    "exception" : re.compile("(?<=\[)\d(?=\]\sPRV)"),
                },
                "columns":['pc','instr','gpr','csr','binary','mode','instr_str','valid','exception'],

    },
    "gen_csv":{
        "process":"python3 "+riscv_dv+"scripts/ovpsim_log_to_trace_csv.py --dont_truncate_after_first_ecall --log ",
        "paths":{
            "log":riscv_dv+"out/ovpsim_sim/"
        },
        "errors":["FileNotFoundError","error: unrecognized arguments:"],

    },
    "gen_cov":{
        "paths":{
            "logs": riscv_dv+"out/ovpsim_sim/",
        },
        "test_param" : "python3 cov.py --iss ovpsim -si riviera --custom_target target/rv64_noelv/",
        "process_param": 'cd '+riscv_dv+' && source ~/.bashrc && module load aldec/riviera/2021.10 && ',
        "test_sucess": "Coverage results are saved",
    },
    "trace_compare":{
        "columns":["Test",'Iteration','Date','GPR Pass','CSR Pass','PC Pass','Binary Pass','Truncation','Verification Status'],

    }



    
}

## Generate RISCV-DV tests and run OVPsim
- Tests that work: 0
- Tests that sometimes timeout but sometimes doesn't: 1, 2, 10
- Tests that doesn't run at all (sometimes): 1, 2, 5, 6, 8, 9 (Never enters program after running the bootloader)
- Test 7 seems to outright fail

In [4]:
def gen_test():
    paths_before_run = glob.glob(settings['gen_test']['paths']["test"])
    srec_after_run = []
    process = Popen(settings['gen_test']['process_param']+settings["gen_test"]["test_param"], shell=True, stdout=PIPE, stderr=PIPE)
    timeouts = []
    stdout, stderr = process.communicate()
    error = stderr.decode('ascii')
    f = open(riscv_dv+'gen_test_output.log','a')
    f.write(error)
    f.close()

    find_error(settings['gen_test']['errors'], error)

    paths_after_run = glob.glob(settings['gen_test']['paths']["test"])
    #print(paths_after_run)
    if(len(paths_after_run) <= len(paths_before_run)):
        raise ValueError("No new tests were generated")

    for path in list(set(paths_after_run)-set(paths_before_run)):
        try:
            f = open(path, "r")
        except IOError:
            print('Warning: File not found '+path)
            continue
        timeouts.append(settings["gen_test"]["arg_timeout"] if f.read().find("Info "+str(finish_after)+":") != -1 else settings["gen_test"]["arg_no_timeout"])
        srec_after_run.append(path[path.find('riscv_'):].rstrip('.log')+'.srec')
        f.close()

    return srec_after_run, timeouts



## Run RTL simulation


In [5]:
def run_rtl(srec_files, timeout_arg):
    if(len(srec_files) != len(timeout_arg)):
        raise ValueError("Lists are not the same length")
    rtl_log_filenames = []
    for srec,timeout in zip(srec_files,timeout_arg):
        name = srec[srec.find('riscv_'):].rstrip('.srec')
        print('Running RTL :'+name)

        try:
            f = open(settings['run_rtl']['paths']['Makefile'],"r")
        except IOError:
            raise RuntimeError('NOEL-V Makefile not found')
        txt = f.read()
        tmp = re.sub("(?<=VSIMOPT=).*",timeout,txt)
        f.close()
        f = open(settings['run_rtl']['paths']['Makefile'],"w")
        f.write(tmp)
        f.close()

        shutil.copy(settings['run_rtl']['paths']['srec']+srec, settings['run_rtl']['paths']['cp_dest']) #copy the current test to the sim dir
        print("Copied {} to the NOELV directory".format(srec))
        #launch sim
        process = Popen(settings['run_rtl']['process_param'], shell=True, stdout=PIPE, stderr=PIPE)
        stdout, stderr = process.communicate()
        rtl_out = stdout.decode('ascii')


        if(find_success(settings['run_rtl']['test_sucess'],rtl_out)):
            print("RTL Simulation Success\n")
        else:
            print("RTL Simulation Failed\n")

        if not os.path.exists(settings['run_rtl']['paths']['rtl_log']):
            os.mkdir(settings['run_rtl']['paths']['rtl_log'])
        name = srec[srec.find('riscv_'):].rstrip('.srec')
        shutil.copy(settings['run_rtl']['paths']['transcript'], settings['run_rtl']['paths']['rtl_log']+name+".rtl.log")
        rtl_log_filenames.append(name+".rtl.log")
    return rtl_log_filenames

## Parse RTL transcript and create rtl_log

In [57]:
def replace_gpr(pd_series,gpr,re_gpr):
    tmp = []
    for line in pd_series:
        split_line = line.split(":")
        if(len(split_line[0]) > 0 and split_line[0] == gpr):
            split_line[0] = re_gpr
        ln = split_line[0]+":"+split_line[1]
        tmp.append(ln)
    return pd.Series(tmp)

def parse_rtl(filenames):
    rtl_data_frames = []
    
    for filename in filenames:
        filename = settings['run_rtl']['paths']['rtl_log']+filename
        try:
            f = open(filename, "r")
        except IOError:
            print('Warning: File not found '+filename)
            continue

        transcript = f.readlines()
        f.close()
        pc,binary,instr_str,gpr,csr,mode,exception,valid = [],[],[],[],[],[],[],[]
        for ln_idx,line in tqdm(enumerate(transcript)):
            if(re.search(settings["parse_rtl"]["reg_ex"]["lookup"],line)): #check that line conforms to format then extract information
                pc.append(re.search(settings['parse_rtl']['reg_ex']["pc"], line).group(0))
                binary.append(re.search(settings['parse_rtl']['reg_ex']["binary"], line).group(0))
                instr_tmp = re.search(settings['parse_rtl']['reg_ex']["instr_str"], line).group(0).strip(" ")
                instr_str.append(instr_tmp)
                csr.append(re.search(settings['parse_rtl']['reg_ex']["csr"],line).group(0).replace(' ','').replace("=0x",":"))
                mode.append(re.search(settings['parse_rtl']['reg_ex']["mode"],line).group(0))
                exception.append(re.search(settings['parse_rtl']['reg_ex']["exception"],line).group(0))
                valid.append(re.search(settings['parse_rtl']['reg_ex']['valid'], line).group(0))
                
                gpr_tmp = re.search(settings['parse_rtl']['reg_ex']["gpr"], line).group(0).replace(" ","").replace("=0x",":")
                if(bool(re.search(settings['parse_rtl']['reg_ex']['isFloat'], instr_tmp) and \
                    bool(re.search(settings['parse_rtl']['reg_ex']['isFloatReg'],gpr_tmp)))\
                ):
                    bound = ln_idx+100 if ln_idx + 100 < len(transcript) else len(transcript)
                    for fpu_line in transcript[ln_idx:bound]:
                        if(bool(re.search(settings['parse_rtl']['reg_ex']['FPU_id'],fpu_line))):
                            if((gpr_tmp[-2:] == re.search(settings['parse_rtl']['reg_ex']['FPU_id'],fpu_line).group(0)) and (gpr_tmp.split(':')[0].find('f') >= 0)):
                                gpr_tmp = re.search(settings['parse_rtl']['reg_ex']['FPU_val'], fpu_line).group(0).replace(" ","").replace("=0x",":")
                                break
                gpr.append(gpr_tmp)


        columns = settings['parse_rtl']['columns']
        rtl_log = pd.DataFrame(columns=columns)
        rtl_log[columns[0]] = pc
        rtl_log[columns[1]] = [x.split(' ')[0] for x in instr_str ]
        rtl_log[columns[2]] = gpr
        rtl_log[columns[3]] = csr
        rtl_log[columns[4]] = binary
        rtl_log[columns[5]] = mode
        rtl_log[columns[6]] = instr_str
        rtl_log[columns[7]] = valid
        rtl_log[columns[8]] = exception
        rtl_log = rtl_log[boot_loader_length:].reset_index(drop=True) #boot_loader_length is where the bootloader ends

        rtl_log.gpr = replace_gpr(rtl_log.gpr, 'fp','s0') #replace fp with s0 to keep convention consistent
        rtl_log['valid'] = rtl_log['valid'].astype('int32') #not true when we have an exception (ecall for instance)
        rtl_log = rtl_log[~((rtl_log['instr'] == "addi") & (rtl_log.shift(1)['instr'] == "mret") & (rtl_log['valid'] == 0))].reset_index(drop=True) #deal with gaislers addi injection after mret, this removes all of those lines
        rtl_log['filename'] = filename.rstrip(".rtl.log")
        rtl_log = rtl_log[~((rtl_log['instr'] == "unknown") & (rtl_log['valid'] == 0) & (rtl_log.shift(1)['valid'] == 0))].reset_index(drop=True)
        rtl_data_frames.append(rtl_log)
    return rtl_data_frames 

In [52]:
line = "#       484906 ns : C0 I0 : 48463    [1] @0x00000000000046b4 (0x261c)     fld fa5, 8(a2)                     WF[fa5 =0x0000000000000018][0] W[ustatus      =0x0000000000016598][0] IPC = 0.2267 Dual = 0.0594 E[cause =0x00] E[tval =0x0000000000000000][0] PRV[3]"
instr_tmp = re.search(settings['parse_rtl']['reg_ex']["instr_str"], line)
if(instr_tmp):
    instr_tmp.strip(" "))
gpr_tmp = re.search(settings['parse_rtl']['reg_ex']["gpr"], line).group(0).replace(" ","").replace("=0x",":")
print(instr_tmp.group(0))
if(bool(re.search(settings['parse_rtl']['reg_ex']['isFloat'], instr_tmp.group(0)) and \
    bool(re.search(settings['parse_rtl']['reg_ex']['isFloatReg'],gpr_tmp)))\
):
    print("cai")
bool(re.search(settings['parse_rtl']['reg_ex']['isFloat'], instr_tmp.group(0)))

SyntaxError: invalid syntax (<ipython-input-52-46f63fe5960f>, line 4)

## Convert OVPsimlog to csv

In [60]:
def convert_sim_log_to_dataframe(filenames):
    #expected .rtl.log extension
    sim_data_frames = []
    for file in filenames:
        print('Converting {} to csv'.format(file.rstrip('rtl.log')+'.log'))
        log = settings['gen_csv']['paths']['log']+file.rstrip('.rtl.log')+'.log'
        csv = settings['gen_csv']['paths']['log']+file.rstrip('.rtl.log')+".csv"
        process = Popen(settings['gen_csv']['process']+log+ " --csv "+csv, shell=True, stdout=PIPE, stderr=PIPE)
        stdout, stderr = process.communicate()
        error = stderr.decode('ascii')
        find_error(settings['gen_csv']['errors'], error)
        time.sleep(1) #try this to make sure that the csv file has time to be closed before being read        sim_log = pd.read_csv(csv)
        sim_log['mode'] = sim_log['mode'].astype('Int64')
        sim_log['gpr'] = sim_log['gpr'].astype('str')
        sim_log['csr'] = sim_log['csr'].astype('str')
        sim_log['filename'] = file.rstrip(".log")
        sim_data_frames.append(sim_log)
    return sim_data_frames


## Compare series to see if run was successful

In [8]:
def truncate_logs(rtl_log,sim_log,no_truncation):
    if(abs(len(sim_log)-len(rtl_log) >= 100)):
        print("Lengths of series differs with "+str(abs(len(sim_log)-len(rtl_log)))+" elements")
        no_truncation = False
    if(len(sim_log) < len(rtl_log)):
        rtl_log=rtl_log.truncate(after=len(sim_log)-1)
        print("Truncating rtl_log to {} elements to meet size of sim_log".format(len(rtl_log)))
    elif(len(sim_log) == len(rtl_log)):
        print("Both logs are equally sized")
    else:
        sim_log=sim_log.truncate(after=len(rtl_log)-1)
        print("Truncating sim_log to {} elements to meet size of rtl_log".format(len(rtl_log)))
    return rtl_log,sim_log,no_truncation

#Truncation
def compare_traces(rtl_data_frames, sim_data_frames):
    if(len(rtl_data_frames) != len(sim_data_frames)):
        raise ValueError('Lists of dataframes are not the same length')
    errors = pd.DataFrame(columns=settings["trace_compare"]["columns"])
    for rtl_log, sim_log in zip(rtl_data_frames,sim_data_frames):
        no_truncation = True
        rtl_log, sim_log, no_truncation = truncate_logs(rtl_log,sim_log,no_truncation)
        #Csr match conditions
        mt_csr_0 = sim_log.csr != "nan"

            
        rtl_csr = rtl_log[(rtl_log.csr.ne(sim_log.csr) & mt_csr_0)].csr #these are the ones we can't exclude directly
        sim_csr = sim_log[rtl_log.csr.ne(sim_log.csr) & mt_csr_0].csr.str.split(";") 
        csr_match_count = compare_csr(sim_csr, rtl_csr)
        mt_csr_1 = csr_information_present(sim_csr,rtl_csr) #ignore csr comparisons where for example mstatus is present the iss but not in the rtl log

        # Since we couldn't configure the iss granularity for PMP we do an unsafe ignore here based on what the operand containing pmpaddr and the instruction being a csr instruction
        # We then check that the gpr content of the instructions only differs by one, however, this could potentially miss some gpr missmatches.
        gpr_pmp_missmatch_tol = 1 #connected to pmp_g in NOEL-V
        rtl_tmp = rtl_log[(sim_log.gpr != 'nan') & (sim_log.operand.str.find('pmpaddr') >= 0) & (sim_log.instr.str.find('csr') >= 0)].gpr
        sim_tmp = sim_log[(sim_log.gpr != 'nan') & (sim_log.operand.str.find('pmpaddr') >= 0) & (sim_log.instr.str.find('csr') >= 0)].gpr
        count = (np.abs(sim_tmp.apply(lambda x: int(x.split(':')[1],16)) - rtl_tmp.apply(lambda x: int(x.split(':')[1],16))) == gpr_pmp_missmatch_tol).value_counts()
        soft_match_gpr = 0
        if(np.where(count.index == True)[0].size > 0):
            soft_match_gpr = count[True]
        matching_gpr = len(sim_log[(sim_log.gpr != "nan") & (rtl_log.gpr == sim_log.gpr)])
        nan_cnt_gpr = len(sim_log[sim_log.gpr == "nan"])
        #these csrs only contain one update to the csr, however, where ovpsim reports more than one update we use csr_match_count
        matching_csr = sim_log[((sim_log.csr != "nan")) & (rtl_log.csr.eq(sim_log.csr))].csr.count() + csr_match_count
        nan_cnt_csr = sim_log[(sim_log.csr == "nan")].csr.count() + len(mt_csr_1)-csr_match_count#[mt_csr_1 == False].count()
        
        tot_cnt_gpr = matching_gpr + nan_cnt_gpr + soft_match_gpr
        gpr_match = (tot_cnt_gpr == len(rtl_log.gpr))
        binary_match = rtl_log.binary.eq(sim_log.binary).all()
        pc_match = rtl_log.pc.eq(sim_log.pc).all()

        csr_match = (matching_csr+nan_cnt_csr == len(rtl_log.csr))
        ig_csr_perc = 1-matching_csr/(rtl_log.csr.count()-sim_log.csr[(sim_log.csr=="nan")].count())

        if(gpr_match and binary_match and pc_match and csr_match and no_truncation):
            print("GPR, Binary, PC and CSR contents match\nIgnored {:.1%} CSR and {:.1%} of GPR comparisons".format(ig_csr_perc,nan_cnt_gpr/rtl_log.gpr.count()))
        else:
            print("PC match: {}\nBinary match: {}\nGPR match: {}\nCSR match: {}\nIgnored {:.1%} CSR and {:.1%} GPR comparisons".format(pc_match,binary_match,gpr_match,csr_match,ig_csr_perc,nan_cnt_gpr/rtl_log.gpr.count()))
            errors = pd.DataFrame(columns=settings["trace_compare"]["columns"])
            name = rtl_log.filename.values[0][rtl_log.filename.values[0].find('riscv_'):]
            print(name)
            errors['Test'] = [name]
            errors['Iteration'] = [re.search('(?<=:\d{2}_)\d*',name).group(0)]
            errors['Date'] = [re.search('\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2}',name).group(0)]
            errors['GPR Pass'] = [gpr_match]
            errors['PC Pass'] = [pc_match]
            errors['Binary Pass'] = [binary_match]
            errors['CSR Pass'] = [csr_match]
            errors['Truncation'] = [not no_truncation]
            # print(errors)
            if not os.path.exists(riscv_dv+'error_log.csv'):
                errors.to_csv(riscv_dv+'error_log.csv',mode='a',header=True,index=False)
            else:
                errors.to_csv(riscv_dv+'error_log.csv',mode='a',header=False,index=False)

In [9]:
#main
for _ in tqdm(range(8)):
    s,t = gen_test()
    rtl_list = run_rtl(s,t)
    rtl_dataframes = parse_rtl(rtl_list)
    sim_dataframes = convert_sim_log_to_dataframe(rtl_list)
    compare_traces(rtl_dataframes,sim_dataframes)


  0%|          | 0/8 [00:00<?, ?it/s]

Running RTL :riscv_invalid_csr_test_2022-03-09_10:38:56_0
Copied riscv_invalid_csr_test_2022-03-09_10:38:56_0.srec to the NOELV directory
RTL Simulation Success

Running RTL :riscv_sfence_exception_test_2022-03-09_10:38:56_0
Copied riscv_sfence_exception_test_2022-03-09_10:38:56_0.srec to the NOELV directory
RTL Simulation Success

Running RTL :riscv_pmp_test_2022-03-09_10:38:56_1
Copied riscv_pmp_test_2022-03-09_10:38:56_1.srec to the NOELV directory
RTL Simulation Success

Running RTL :riscv_floating_point_arithmetic_test_2022-03-09_10:38:56_0
Copied riscv_floating_point_arithmetic_test_2022-03-09_10:38:56_0.srec to the NOELV directory
RTL Simulation Success

Running RTL :riscv_hint_instr_test_2022-03-09_10:38:56_1
Copied riscv_hint_instr_test_2022-03-09_10:38:56_1.srec to the NOELV directory
RTL Simulation Success

Running RTL :riscv_loop_test_2022-03-09_10:38:56_1
Copied riscv_loop_test_2022-03-09_10:38:56_1.srec to the NOELV directory
RTL Simulation Success

Running RTL :riscv_sfe


0it [00:00, ?it/s]
6799it [00:00, 67967.05it/s]
13596it [00:00, 41354.82it/s]
18304it [00:00, 36519.33it/s]
22251it [00:00, 34505.20it/s]
25851it [00:00, 33748.16it/s]
29306it [00:00, 33320.62it/s]
32684it [00:00, 33033.91it/s]
36015it [00:01, 32813.88it/s]
39313it [00:01, 32625.93it/s]
42586it [00:01, 32531.42it/s]
45846it [00:01, 32428.00it/s]
49093it [00:01, 32275.55it/s]
52323it [00:01, 32231.41it/s]
55548it [00:01, 32153.97it/s]
58765it [00:01, 32121.89it/s]
61985it [00:01, 32143.53it/s]
65200it [00:01, 32125.55it/s]
68413it [00:02, 32117.56it/s]
71625it [00:02, 32004.22it/s]
74843it [00:02, 32055.87it/s]
78049it [00:02, 32043.07it/s]
81254it [00:02, 31963.18it/s]
84459it [00:02, 31987.24it/s]
87671it [00:02, 32025.26it/s]
90879it [00:02, 32039.94it/s]
94105it [00:02, 32103.61it/s]
97316it [00:02, 32094.98it/s]
100539it [00:03, 32134.26it/s]
104633it [00:03, 33113.29it/s]

0it [00:00, ?it/s]
6846it [00:00, 68444.89it/s]
13691it [00:00, 41665.84it/s]
18434it [00:00, 37763.64it/s]


88620it [00:02, 32346.51it/s]
91855it [00:02, 32331.08it/s]
95099it [00:02, 32361.73it/s]
98336it [00:02, 32313.86it/s]
104650it [00:03, 33485.28it/s]

0it [00:00, ?it/s]
7508it [00:00, 62191.34it/s]

0it [00:00, ?it/s]
6810it [00:00, 68078.32it/s]
13618it [00:00, 40795.25it/s]
18296it [00:00, 36929.50it/s]
22286it [00:00, 35122.21it/s]
25951it [00:00, 33946.83it/s]
29425it [00:00, 33303.23it/s]
32798it [00:00, 32824.47it/s]
36103it [00:01, 32466.32it/s]
39362it [00:01, 32178.44it/s]
42586it [00:01, 31918.73it/s]
45781it [00:01, 31781.72it/s]
48961it [00:01, 31700.38it/s]
52132it [00:01, 31655.70it/s]
55298it [00:01, 31573.54it/s]
58456it [00:01, 31473.07it/s]
61604it [00:01, 31442.63it/s]
64760it [00:01, 31476.30it/s]
67918it [00:02, 31506.07it/s]
71069it [00:02, 31452.70it/s]
74216it [00:02, 31457.34it/s]
77362it [00:02, 31428.75it/s]
80507it [00:02, 31428.25it/s]
83658it [00:02, 31450.96it/s]
86808it [00:02, 31464.56it/s]
89955it [00:02, 31400.06it/s]
93111it [00:02, 31445.81it/s]
9

80643it [00:02, 31547.68it/s]
83798it [00:02, 31519.94it/s]
86956it [00:02, 31536.53it/s]
90110it [00:02, 31513.28it/s]
93277it [00:02, 31558.87it/s]
96433it [00:02, 31544.59it/s]
99588it [00:03, 31494.95it/s]
104650it [00:03, 32704.00it/s]

0it [00:00, ?it/s]
6791it [00:00, 67896.47it/s]
13581it [00:00, 40954.42it/s]
18263it [00:00, 36987.39it/s]
22254it [00:00, 35060.13it/s]
25909it [00:00, 34031.81it/s]
29390it [00:00, 33378.28it/s]
32770it [00:00, 32854.38it/s]
36078it [00:01, 32463.83it/s]
39336it [00:01, 32200.55it/s]
42562it [00:01, 32033.64it/s]
45769it [00:01, 31890.31it/s]
48960it [00:01, 31796.42it/s]
52141it [00:01, 31624.11it/s]
55304it [00:01, 31568.09it/s]
58466it [00:01, 31581.17it/s]
61625it [00:01, 31546.37it/s]
64780it [00:01, 31544.59it/s]
67935it [00:02, 31505.45it/s]
71086it [00:02, 31497.71it/s]
74243it [00:02, 31518.65it/s]
77395it [00:02, 31507.20it/s]
80550it [00:02, 31514.40it/s]
83702it [00:02, 31384.19it/s]
86841it [00:02, 31329.56it/s]
89988it [00:02, 3137

...

18462it [00:00, 37641.85it/s]
22519it [00:00, 35845.16it/s]
26254it [00:00, 34807.42it/s]
29814it [00:00, 34091.78it/s]
33266it [00:00, 33618.60it/s]
36651it [00:01, 33253.08it/s]
39989it [00:01, 32975.58it/s]
43293it [00:01, 32714.12it/s]
46568it [00:01, 32487.05it/s]
49818it [00:01, 32322.30it/s]
53051it [00:01, 32204.27it/s]
56272it [00:01, 32173.31it/s]
59490it [00:01, 32156.27it/s]
62708it [00:01, 32160.04it/s]
65942it [00:01, 32211.39it/s]
69164it [00:02, 32187.09it/s]
72387it [00:02, 32198.24it/s]
75607it [00:02, 32093.09it/s]
78823it [00:02, 32110.31it/s]
82055it [00:02, 32171.37it/s]
85273it [00:02, 32018.39it/s]
88495it [00:02, 32076.36it/s]
91720it [00:02, 32126.06it/s]
94941it [00:02, 32149.42it/s]
98157it [00:02, 32130.48it/s]
101371it [00:03, 32122.22it/s]
104633it [00:03, 33356.57it/s]

0it [00:00, ?it/s]
6850it [00:00, 68496.47it/s]
13700it [00:00, 41586.62it/s]
18440it [00:00, 37742.98it/s]
22503it [00:00, 35884.35it/s]
26239it [00:00, 34822.79it/s]
29799it [00:00, 340

Converting riscv_hint_instr_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_sfence_exception_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_machine_mode_rand_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_rand_instr_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_invalid_csr_test_2022-03-10_00:33:51_1.rtl.log to csv
Converting riscv_ebreak_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_amo_test_2022-03-10_00:33:51_1.rtl.log to csv
Converting riscv_rand_jump_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_unaligned_load_store_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_privileged_mode_rand_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_mmu_stress_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_full_interrupt_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_ebreak_debug_mode_test_2022-03-10_00:33:51_0.rtl.log to csv
Converting riscv_no_fence_test_2022-03-10_00:33:51_0.rtl.

False    734
Name: gpr, dtype: int64
GPR, Binary, PC and CSR contents match
Ignored 33.3% CSR and 72.5% of GPR comparisons
Truncating rtl_log to 99265 elements to meet size of sim_log
True    734
Name: gpr, dtype: int64
GPR, Binary, PC and CSR contents match
Ignored 33.2% CSR and 72.5% of GPR comparisons
Truncating rtl_log to 99248 elements to meet size of sim_log
True    750
Name: gpr, dtype: int64
GPR, Binary, PC and CSR contents match
Ignored 33.2% CSR and 74.2% of GPR comparisons
Truncating rtl_log to 99248 elements to meet size of sim_log
False    750
Name: gpr, dtype: int64
GPR, Binary, PC and CSR contents match
Ignored 33.2% CSR and 74.2% of GPR comparisons
Truncating rtl_log to 12142 elements to meet size of sim_log
Series([], Name: gpr, dtype: int64)
GPR, Binary, PC and CSR contents match
Ignored 25.0% CSR and 41.4% of GPR comparisons
Truncating rtl_log to 99248 elements to meet size of sim_log
False    750
Name: gpr, dtype: int64
GPR, Binary, PC and CSR contents match
Ignored

100%|██████████| 8/8 [15:59:08<00:00, 7193.55s/it]  

True    750
Name: gpr, dtype: int64
GPR, Binary, PC and CSR contents match
Ignored 33.2% CSR and 74.2% of GPR comparisons
Truncating rtl_log to 31157 elements to meet size of sim_log
Series([], Name: gpr, dtype: int64)
GPR, Binary, PC and CSR contents match
Ignored 0.9% CSR and 41.9% of GPR comparisons


In [23]:
rtl_dataframes = parse_rtl(rtl_list[0])
sim_dataframes = convert_sim_log_to_dataframe(rtl_list[0])
compare_traces(rtl_dataframes,sim_dataframes)

16205it [00:00, 41363.69it/s]
21237it [00:00, 34246.11it/s]
104650it [00:03, 33219.47it/s]
10561it [00:00, 48065.17it/s]
19914it [00:00, 39462.85it/s]
104633it [00:03, 33456.96it/s]
104633it [00:03, 32006.68it/s]
104633it [00:03, 32749.68it/s]
16132it [00:00, 40992.30it/s]
104633it [00:03, 32919.41it/s]
104650it [00:03, 32120.66it/s]
104650it [00:03, 32631.05it/s]
104650it [00:03, 32558.98it/s]
104650it [00:03, 32981.30it/s]
104650it [00:03, 32507.55it/s]
104650it [00:03, 32531.50it/s]
20862it [00:00, 37602.66it/s]
104633it [00:03, 32752.71it/s]
26000it [00:00, 36912.43it/s]
104650it [00:03, 32830.75it/s]
6864it [00:00, 65485.02it/s]
26770it [00:00, 36698.39it/s]
104650it [00:03, 32021.00it/s]
4172it [00:00, 235171.44it/s]
104650it [00:03, 32097.50it/s]
104650it [00:03, 32573.78it/s]
104650it [00:03, 32670.12it/s]
17364it [00:00, 34666.97it/s]
18613it [00:00, 39134.20it/s]
104633it [00:03, 32602.28it/s]
9476it [00:00, 50076.07it/s]
29058it [00:00, 32627.64it/s]
8638it [00:00, 52440.41i

Converting riscv_arithmetic_basic_test_2022-03-08_23:19:27_1.rtl.log to csv
Converting riscv_floating_point_rand_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_non_compressed_instr_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_machine_mode_rand_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_full_interrupt_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_no_fence_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_rand_jump_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_floating_point_mmu_stress_test_2022-03-08_23:19:27_1.rtl.log to csv
Converting riscv_arithmetic_basic_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_ebreak_debug_mode_test_2022-03-08_23:19:27_1.rtl.log to csv
Converting riscv_loop_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_hint_instr_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_privileged_mode_rand_test_2022-03-08_23:19:27_0.rtl.log to csv
Converting riscv_invalid_

False    751
Name: gpr, dtype: int64
25653
73595
0
99248
GPR, Binary, PC and CSR contents match
Ignored 33.3% CSR and 74.2% of GPR comparisons
Truncating rtl_log to 99248 elements to meet size of sim_log
True    750
Name: gpr, dtype: int64
24903
73595
750
99248
GPR, Binary, PC and CSR contents match
Ignored 33.2% CSR and 74.2% of GPR comparisons
Truncating rtl_log to 10369 elements to meet size of sim_log
Series([], Name: gpr, dtype: int64)
5875
4494
0
10369
GPR, Binary, PC and CSR contents match
Ignored 2.0% CSR and 43.3% of GPR comparisons
Truncating rtl_log to 14716 elements to meet size of sim_log
Series([], Name: gpr, dtype: int64)
9379
5337
0
14716
GPR, Binary, PC and CSR contents match
Ignored 0.4% CSR and 36.3% of GPR comparisons
Truncating rtl_log to 99265 elements to meet size of sim_log
False    734
Name: gpr, dtype: int64
27285
71980
0
99265
GPR, Binary, PC and CSR contents match
Ignored 33.2% CSR and 72.5% of GPR comparisons
Truncating rtl_log to 5579 elements to meet size

In [52]:
#rtl_log[((rtl_log['instr'] == "unknown") & (rtl_log['valid'] == 0) & (rtl_log.shift(1)['valid'] == 0))]
rtl_log

,pc,instr,gpr,csr,binary,mode,instr_str,valid,exception,filename
0,000000000000e2dc,unknown,s4:0000000000000000,ustatus:0000000000000000,be397a5f,3,unknown instruction,0,1,/home/matteo/MasterThesis/riscv-dv/out/rtl_log...
1,000000000000e2dc,unknown,s4:0000000000000000,ustatus:0000000000000000,be397a5f,3,unknown instruction,0,1,/home/matteo/MasterThesis/riscv-dv/out/rtl_log...


In [45]:
rtl_log[rtl_log["pc"]=="000000000000cbe6"]

,pc,instr,gpr,csr,binary,mode,instr_str,valid,exception,filename
11025,000000000000cbe6,unknown,t5:0000000000000000,ustatus:0000000000000000,c64e1f6b,3,unknown instruction,0,1,/home/matteo/MasterThesis/riscv-dv/out/rtl_log...
22449,000000000000cbe6,unknown,t5:0000000000000000,ustatus:0000000000000000,c64e1f6b,3,unknown instruction,0,1,/home/matteo/MasterThesis/riscv-dv/out/rtl_log...


In [ ]:
# Use this to fetch the rtl list again from the path of rtl_log
rtl_logs = glob.glob(riscv_dv+'out/rtl_log/*')
rtl_list = [bool(re.search("^((?!12:29:42).)*$",x)) for x in rtl_logs]
rtl_list = list(compress(rtl_logs,rtl_list))
rtl_list = [x[x.find('riscv_'):] for x in rtl_list]
rtl_dataframes = parse_rtl(rtl_list)
sim_dataframes = convert_sim_log_to_dataframe(rtl_list)
compare_traces(rtl_dataframes,sim_dataframes)

## Diagnose potential problems put into the error log

In [59]:
#What are the errors? 
errors = pd.read_csv(riscv_dv+'error_log.csv')
num = 12-2 #specify row of csv to display errors
test = errors[num:num+1]
name = [test.Test.values[0]+'.rtl.log']
rtl_log = parse_rtl(name)
sim_log = convert_sim_log_to_dataframe(name)
rtl_log,sim_log,_ = truncate_logs(rtl_log[0],sim_log[0],False)
if(not test['GPR Pass'].bool()):
    print("GPR issue")
    print(np.unique(rtl_log[(rtl_log.gpr != sim_log.gpr) & (sim_log.gpr != 'nan') & ~((sim_log.gpr != 'nan') & (sim_log.operand.str.find('pmpaddr') >= 0) & (sim_log.instr.str.find('csr') >= 0))].gpr))
    print(np.unique(sim_log[(rtl_log.gpr != sim_log.gpr) & (sim_log.gpr != 'nan') & ~((sim_log.gpr != 'nan') & (sim_log.operand.str.find('pmpaddr') >= 0) & (sim_log.instr.str.find('csr') >= 0))].gpr))
if(not test['CSR Pass'].bool()):
    print("CSR issue")
    print(rtl_log[(rtl_log.csr != sim_log.csr) & (sim_log.csr != 'nan') & (rtl_log.csr != 'ustatus:0000000000000000')].csr)
    print(sim_log[(rtl_log.csr != sim_log.csr) & (sim_log.csr != 'nan') & (rtl_log.csr != 'ustatus:0000000000000000')].csr)
if(not test['Binary Pass'].bool()):
    print("OPcode issue")
    print(rtl_log[(rtl_log['binary'] != sim_log['binary']) & (sim_log['binary'] != 'nan')]['binary'])
    print(sim_log[(rtl_log['binary'] != sim_log['binary']) & (sim_log['binary'] != 'nan')]['binary'])
if(not test['PC Pass'].bool()):
    print("PC issue")
    print(rtl_log[(rtl_log.pc != sim_log.pc) & (sim_log.pc != 'nan')].pc)
    print(sim_log[(rtl_log.pc != sim_log.pc) & (sim_log.pc != 'nan')].pc)


25737it [00:00, 37930.11it/s]


Converting riscv_floating_point_mmu_stress_test_2022-03-09_14:36:25_0.rtl.log to csv
Truncating rtl_log to 18978 elements to meet size of sim_log
GPR issue
[]
[]


In [16]:
rtl_log[10945:10946]

,pc,instr,gpr,csr,binary,mode,instr_str,valid,exception,filename
10945,00000000000046b4,fld,fa5:0000000000000018,ustatus:0000000000016598,261c,3,"fld fa5, 8(a2)",1,0,/home/matteo/MasterThesis/riscv-dv/out/rtl_log...


In [57]:
sim_log[5485:5495]

,pc,instr,gpr,csr,binary,mode,instr_str,operand,pad,filename
5485,0000000000007996,fsw,nan,nan,fe9b21a7,3,"fsw fs1,-29(s6)","fs1,s6,-29",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5486,000000000000799a,sb,nan,nan,03cb0023,3,"sb t3,32(s6)","t3,s6,32",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5487,000000000000799e,sh,nan,nan,ffab16a3,3,"sh s10,-19(s6)","s10,s6,-19",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5488,00000000000079a2,remuw,s2:0000000000000000,nan,02a8793b,3,"remuw s2,a6,a0","s2,a6,a0",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5489,00000000000079a6,fmin.s,nan,nan,29e28cd3,3,"fmin.s fs9,ft5,ft10","fs9,ft5,ft10",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5490,00000000000079aa,fsqrt.d,ft9:5ff0000000000000,nan,5a09fed3,3,"fsqrt.d ft9,fs3","ft9,fs3",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5491,00000000000079ae,sb,nan,nan,ffcb0f23,3,"sb t3,-2(s6)","t3,s6,-2",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5492,00000000000079b2,auipc,t5:00000000000179b2,nan,00010f17,3,"auipc t5,0x10","t5,0x10",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5493,00000000000079b6,addi,t5:00000000000181a1,nan,7eff0f13,3,"addi t5,t5,2031","t5,t5,2031",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...
5494,00000000000079ba,lh,nan,nan,98df1703,3,"lh a4,-1651(t5)","a4,t5,-1651",NaN,riscv_floating_point_mmu_stress_test_2022-03-0...


## Run the next cell to disasemble the program in which you had an error

In [ ]:
objdump = os.environ['RISCV_OBJDUMP']
settings = '-d'
program = riscv_dv+'out/asm_test/'+test.Test.values[0]+'.o'

process = Popen('{} {} {}'.format(objdump,settings,program),stdout=PIPE, shell=True)
stdout, stderr = process.communicate()
disas_program = stdout.decode('utf-8')
print(disas_program)

In [ ]:
## Coverage results

path = glob.glob(settings["gen_cov"]["paths"]["logs"])
process = Popen(settings["gen_cov"]["process_param"]+settings["gen_cov"]["test_param"]+" --dir "+path[0], shell=True, stdout=PIPE, stderr=PIPE)

stdout, stderr = process.communicate()
cov_out = stdout.decode('ascii')
cov_err = stderr.decode('ascii')
if(find_success(settings['gen_cov']['test_sucess'],cov_err)):
    print("Coverage results produced successfully")
else:
    print(cov_err)
